In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
#NASA Mars News
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [5]:
newshtml = browser.html
soup = bs(newshtml, 'html.parser')
page = soup.find_all("div", {"id": "page"})
page

[<div id="page">
 <!-- title to go in the page_header -->
 <div class="header_mask">
 <section class="content_page module"></section>
 </div>
 <div class="grid_list_page module content_page">
 <div class="grid_layout">
 <article>
 <header id="page_header">
 </header>
 <div class="react_grid_list" data-react-class="GridListPage" data-react-props='{"left_column":false,"class_name":"","default_view":"list_view","model":"news_items","view_toggle":false,"search":"true","list_item":"News","title":"News","categories":["19,165,184,204"],"order":"publish_date desc,created_at desc","no_items_text":"There are no items matching these criteria.","site_title":"NASA’s Mars Exploration Program ","short_title":"Mars","site_share_image":"/system/site_config_values/meta_share_images/1_142497main_PIA03154-200.jpg","per_page":null,"filters":"[ [ \"date\", [ [ \"2019\", \"2019\" ], [ \"2018\", \"2018\" ], [ \"2017\", \"2017\" ], [ \"2016\", \"2016\" ], [ \"2015\", \"2015\" ], [ \"2014\", \"2014\" ], [ \"201

In [6]:
#test = soup.find_all("h3")
#for each in test:
#    print(each)

#above works because find will only find the first one, which is the most recent -- same for the body, below
news_title = soup.find("h3").text
news_title

'After a Reset, Curiosity Is Operating Normally'

In [9]:
news_p = soup.find("div",class_="rollover_description_inner").text
news_p

"NASA's Mars rover Curiosity is in good health but takes a short break while engineers diagnose why it reset its computer. "

In [33]:
#JPL Mars Space Images - Featured Image

url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [34]:
jplhtml = browser.html
soup = bs(jplhtml, 'html.parser')

In [35]:
#carouselPic = soup.find('article',class_='carousel_item')
#image url is inside an article tag in the 'style' parameter, which is why
#we're accessing it below like so
imageExt = soup.article['style'][23:-3]
featured_image_url = 'https://www.jpl.nasa.gov' + imageExt
featured_image_url

In [36]:
#Mars Weather
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)

In [37]:
twitterhtml = browser.html
soup = bs(twitterhtml, 'html.parser')

In [38]:
#Find the tweets
tweets = soup.find_all("p", class_ = "TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")

In [50]:
#Get the first tweet
count = 1
recentTweets = []
for tweet in tweets:
    #make sure it's a weather tweet
    if "pressure at" in tweet.text or "winds from the" in tweet.text:
        startIdx = tweet.text.lower().index("sol")
        if "\n" in tweet.text:
            endIdx = tweet.text.index("\n")
            recentTweets.append(tweet.text[startIdx:endIdx])
            if count == 1:
                mars_weather = tweet.text[startIdx:endIdx]
                count +=1
        else:
            recentTweets.append(tweet.text[startIdx:])
            if count == 1:
                mars_weather = tweet.text[startIdx:]
                count +=1
print(mars_weather)
recentTweets

sol 84 (2019-02-20) low -95.1ºC (-139.2ºF) high -13.2ºC (8.3ºF)


['sol 84 (2019-02-20) low -95.1ºC (-139.2ºF) high -13.2ºC (8.3ºF)',
 'sol 83 (2019-02-19) low -95.0ºC (-139.0ºF) high -11.7ºC (11.0ºF)',
 'sol 82 (2019-02-18) low -94.8ºC (-138.6ºF) high -12.3ºC (9.8ºF)',
 'sol 81 (2019-02-17), high -17/2F, low -95/-138F, pressure at 7.23hPa, winds from the WNW at 12 mph gusting to 37.8 mph',
 'sol 80 (2019-02-16), high -16/3F, low -95/-139F, pressure at 7.23hPa, winds from the WNW at 10.7 mph gusting to 32.3 mph',
 'sol 79 (2019-02-15), high -13/8F, low -95/-138F, pressure at 7.21hPa, winds from the WNW at 9.9 mph gusting to 26.2 mph',
 'sol 78 (2019-02-14), high -15/4F, low -95/-138F, pressure at 7.22hPa, winds from the WNW at 10.5 mph gusting to 36.4 mph',
 'sol 77 (2019-02-13), high -13C/10F, low -94C/-136F, pressure at 7.21 hPa, winds from the WNW at 9 mph gusting to 26.6 mph',
 'Sol 2319 (2019-02-13), high -17C/1F, low -72C/-97F, pressure at 8.12 hPa, daylight 06:46-18:52pic.twitter.com/anlHR95BMs',
 'Sol 2318 (2019-02-12), high -17C/1F, low -73C

In [40]:
#Mars Facts
url = "https://space-facts.com/mars/"
browser.visit(url)

In [41]:
factshtml = browser.html
soup = bs(factshtml, 'html.parser')

In [42]:
table = soup.find_all("table", {'id':'tablepress-mars'})

io = ""
for item in table:
    io = io + str(item)
    
marslist = pd.read_html(io)
#choose the dataframe from list of dataframes
df = marslist[0]
#rename the '0' and '1' columns with meaningful titles
df = df.rename(columns = {0:'Description', 1:'Value'}).set_index('Description')

#make an html string to represent the dataframe
dfhtml = df.to_html()
dfhtml

In [48]:
#Mars Hemispheres
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [44]:
hemhtml = browser.html
soup = bs(hemhtml, 'html.parser')

In [45]:
imgUrls = []
links = soup.find_all("div", class_="item")
for link in links:
    imglink = "https://astrogeology.usgs.gov" + link.a['href']
    imgUrls.append(imglink)
imgUrls

[]

In [46]:
#visit each of the image links to get the image url and hemisphere name
hemisphere_image_urls = []
for idx, val in enumerate(imgUrls):
    url = val
    browser.visit(url)
    
    imghtml = browser.html
    soup = bs(imghtml, 'html.parser')
    
    imgs = soup.find_all("img", class_ = "wide-image")
    name = soup.find("h2",class_="title")
    
    for content in imgs:
        img_url = 'https://astrogeology.usgs.gov' + content["src"]
    
    #append a dictionary to list of dicts for each image
    hemisphere_image_urls.append({'title': name.text[:-9], 'img_url' : img_url})
    
    #print what the contents of the dictionary will look like
    print(name.text[:-9])
    print(img_url)
    print("--------------------------------------------------------")

In [47]:
ultimate = {
    'newsTitle':news_title,
    'newsPara':news_p,
    'featuredImg':featured_image_url,
    'marsWeather':mars_weather,
    'marsTable':dfhtml,
    'hemispheres':hemisphere_image_urls
}
ultimate

{'newsTitle': 'InSight Is the Newest Mars Weather Service',
 'newsPara': "By collecting data around the clock, NASA's lander will provide unique science about the Martian surface.",
 'featuredImg': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19343-1920x1200.jpg',
 'marsWeather': 'sol 84 (2019-02-20) low -95.1ºC (-139.2ºF) high -13.2ºC (8.3ºF)',
 'marsTable': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 k